In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report,confusion_matrix
%matplotlib inline

In [2]:
def loaddata():
    col = ['ID Number','Diagnosis','radius','texture','perimeter','area','smoothness','compactness','concavity','concave points','symmetry','fractal dimension'
        ,'radius2','texture2','perimeter2','area2','smoothness2','compactness2','concavity2','concave points2','symmetry2','fractal dimension2'
        ,'radius3','texture3','perimeter3','area3','smoothness3','compactness3','concavity3','concave points3','symmetry3','fractal dimension3']

    data = pd.read_csv('wdbc.data',error_bad_lines=False)
    data.columns = col
    return data

In [3]:
def printaccuracy(y_test,predict,model):
    print(model," report")
    print("-------------------------------------")
    print(" ")
    print(confusion_matrix(y_test,predict))
    print(classification_report(y_test,predict))
    print(" ")
    print("-------------------------------------")
    print(" ")

In [4]:
def normalizedata(X):
    SS = StandardScaler()
    X = SS.fit_transform(X)
    print("Normalization done")
    

In [5]:
def PC(components,x):
    cols = []
    pca = PCA(n_components=components)
    pc = pca.fit_transform(x)
    for i in range(components):
        cols.append('pc'+str(i))
    pc_data = pd.DataFrame(data = pc, columns = cols)
    return pc_data

In [6]:
def removeoutliers(data,inplace=False):
    prev_rows = len(data)
    data_copy = data.copy()
    z_score = np.abs(stats.zscore(data_copy))
    data_copy = data_copy[(z_score < 3).all(axis=1)]
    if inplace:
        data=data_copy
    #print("Before removing outliers , rows - ", prev_rows)
    #print("After removing outliers , rows -", len(data_copy))
    #print("Number of records deleted - ", (prev_rows - len(data_copy))#)

In [7]:
def validatecols(data):
    if len(data.columns) == len(col):
        return True
    else:
        return False

In [8]:
def validatedatatypes(trained, newdata):
    for i in range(trained.columns):
        if trained[trained.columns[i]] != newdata[newdata.columns[i]]:
            return False
    return True    

In [9]:
def preprocess():
    data.drop('ID Number',axis=1,inplace=True)
    data['Diagnosis'] = pd.Categorical(data['Diagnosis']).codes
    removeoutliers(data,inplace=True)
    X = data.drop('Diagnosis',axis=1)
    X_copy = X.copy()
    y= data['Diagnosis']
    normalizedata(X)
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)
    return X_train,X_test,y_train,y_test,X_copy,y

In [10]:
def k_vs_error_graph():
    knn_error = []
    for i in range(2,50):
        knn = KNeighborsClassifier(n_neighbors=i)
        knn.fit(X_train,y_train)
        knn_predict= knn.predict(X_test)
        knn_error.append(np.mean(y_test!=knn_predict))
    plt.plot(range(2,50),knn_error)
    plt.xlabel("K value")
    plt.ylabel("Error")

In [11]:
def logisticregression():
    lr = LogisticRegression(solver='lbfgs',max_iter=10000,random_state=0)
    lr.fit(X_train,y_train)
    lr_predict = lr.predict(X_test)
    printaccuracy(y_test,lr_predict,"Logistic Regression")
    return f1_score(y_test,lr_predict)

In [12]:
def KNN():
    neighbors={'n_neighbors':np.array(range(2,50))}
    knn_grid=GridSearchCV(KNeighborsClassifier(),neighbors,verbose=False,refit=True,cv=3)
    knn_grid.fit(X_train,y_train)
    #knn_grid.best_params_
    knn_predict = knn_grid.predict(X_test)
    printaccuracy(y_test,knn_predict,"KNN")
    return f1_score(y_test,knn_predict)

In [13]:
def SVM():
    svm = SVC(kernel='rbf',random_state=0)
    params = { 'C' : np.logspace(0, 3, 4), 'gamma' : np.logspace(-2, 1, 4)}
    svm_grid = GridSearchCV(svm, params,cv=3,verbose=False,return_train_score=True)
    svm_X = X.copy()
    svm_X = PC(2,svm_X)
    svm_X_train,svm_X_test,svm_y_train,svm_y_test=train_test_split(svm_X,y,test_size=0.3)
    svm_grid.fit(svm_X_train,svm_y_train)
    svm_predict = svm_grid.predict(svm_X_test)
    printaccuracy(svm_y_test,svm_predict,"SVM")
    return f1_score(svm_y_test,svm_predictict)

In [14]:
def DecisionTree():
    dt = DecisionTreeClassifier(random_state=0)
    dt.fit(X_train,y_train)
    dt_predict = dt.predict(X_test)
    printaccuracy(y_test,dt_predict,"Decision Tree")
    return f1_score(y_test,dt_predict)

In [15]:
def RandomForest():
    rf = RandomForestClassifier(random_state=0)
    params = { 'n_estimators' : np.arange(10,100,10), 'max_depth' : np.arange(5,50,5)}
    rf_grid = GridSearchCV(rf, params, verbose=False, cv=3)
    rf_grid.fit(X_train,y_train)
    rf_predict = rf_grid.predict(X_test)
    printaccuracy(y_test,rf_predict,"Random Forest")
    rf
    return f1_score(y_test,rf_predict)

In [16]:
def Adaboost():
    ab = AdaBoostClassifier(random_state=0)
    params = { 'n_estimators' : np.arange(10,100,10)}
    ab_grid = GridSearchCV(ab, params, verbose=False, cv=3)
    ab_grid.fit(X_train,y_train)
    ab_predict = ab_grid.predict(X_test)
    printaccuracy(y_test,ab_predict,"Adaboost")
    return f1_score(y_test,ab_predict)

In [17]:
def GaussionNB():
    gnb = GaussianNB()
    gnb.fit(X_train,y_train)
    gnb_predict = gnb.predict(X_test)
    printaccuracy(y_test,gnb_predict,"GaussionNB")
    return f1_score(y_test,gnb_predict)

In [18]:
def NueralNetwork():
    nn = MLPClassifier(solver='sgd',random_state=0)
    params = {
    'hidden_layer_sizes': np.arange(50,150,20),
    'learning_rate': ['constant','adaptive'],
    'max_iter': np.arange(200,300,50)
    }
    #'hidden_layer_sizes': [(100,50), (50,20), (20,10)],
    #'hidden_layer_sizes': np.arange(10,100,20)
    # 'activation': ['tanh', 'relu'],
    # 'alpha': 10.0 ** -np.arange(1, 5),
    #'solver': ['sgd', 'adam'],
    nn_grid = GridSearchCV(nn, params, cv=3,verbose=False)
    nn_grid.fit(X_train,y_train)
    nn_predict = nn_grid.predict(X_test)
    printaccuracy(y_test,nn_predict,"Nueral Network")
    return f1_score(y_test,nn_predict)

In [19]:
def EvaluateModels(f1scores):
    models=['Logistic Regression',
            'K Nearest Neighbours',
            'Support Vector Machine',
            'Decision Tree',
            'Random Forest',
            'AdaBoost',
            'Guassian Naive Bayes',
            'Nueral Network'
           ]

    print("        Model Results    ")
    print(" --------------------------- ")
    for i in range(len(f1scores)):
        print(models[i]," : f1 score - ",f1scores[i])
    print(" -----------------------------")
    print("Best model")
    idx = f1scores.index(np.max(f1scores))
    print(models[idx] ," : f1 score - ", np.max(f1scores))
    
    
    
    
    

In [20]:
def trainmodels():
    f1scores=[]
    f1scores.append(logisticregression())
    f1scores.append(KNN())
    f1scores.append(SVM())
    f1scores.append(DecisionTree())
    f1scores.append(RandomForest())
    f1scores.append(Adaboost())
    f1scores.append(GaussionNB())
    f1scores.append(NueralNetwork())
    return f1scores

In [89]:
dt = np.ones((8,5))
df = pd.DataFrame(dt)

In [98]:
A=['Diabetic Retinopaty','Diabetic Retinopaty','Diabetic Retinopaty','Diabetic Retinopaty','Diabetic Retinopaty','Diabetic Retinopaty','Diabetic Retinopaty','Diabetic Retinopaty']
E=['B','B','B','B','B','B','B','B']
B =['Logistic Regression','KNN','SVM','Decision Tree','Radom Forest','Adaboost','GaussionNB','Nueral Network']
C=['Train Accuracy','Test Accuracy','Precision','Recall','F1 score']

In [92]:
df.index = [A,B]
df.columns=C

In [96]:
df1 = df.copy()

In [99]:
df1.index = [E,B]
df1.columns=C

In [100]:
df2 = pd.concat([df,df1])

In [101]:
df2

Train Accuracy  Test Accuracy  \
Diabetic Retinopaty Logistic Regression             1.0            1.0   
                    KNN                             1.0            1.0   
                    SVM                             1.0            1.0   
                    Decision Tree                   1.0            1.0   
                    Radom Forest                    1.0            1.0   
                    Adaboost                        1.0            1.0   
                    GaussionNB                      1.0            1.0   
                    Nueral Network                  1.0            1.0   
B                   Logistic Regression             1.0            1.0   
                    KNN                             1.0            1.0   
                    SVM                             1.0            1.0   
                    Decision Tree                   1.0            1.0   
                    Radom Forest                    1.0            1.0   
                    Adaboost                        1.0            1.0   
                    GaussionNB                      1.0            1.0   
                    Nueral Network                  1.0            1.0   

                                         Precision  Recall  F1 score  
Diabetic Retinopaty Logistic Regression        1.0     1.0       1.0  
                    KNN                        1.0     1.0       1.0  
                    SVM                        1.0     1.0       1.0  
                    Decision Tree              1.0     1.0       1.0  
                    Radom Forest               1.0     1.0       1.0  
                    Adaboost                   1.0     1.0       1.0  
                    GaussionNB                 1.0     1.0       1.0  
                    Nueral Network             1.0     1.0       1.0  
B                   Logistic Regression        1.0     1.0       1.0  
                    KNN                        1.0     1.0       1.0  
                    SVM                        1.0     1.0       1.0  
                    Decision Tree              1.0     1.0       1.0  
                    Radom Forest               1.0     1.0       1.0  
                    Adaboost                   1.0     1.0       1.0  
                    GaussionNB                 1.0     1.0       1.0  
                    Nueral Network             1.0     1.0       1.0

In [108]:
temp = df2.xs('Diabetic Retinopaty')
temp[temp['F1 score'] == temp['F1 score'].max()]

,Train Accuracy,Test Accuracy,Precision,Recall,F1 score
Logistic Regression,1.0,1.0,1.0,1.0,1.0
KNN,1.0,1.0,1.0,1.0,1.0
SVM,1.0,1.0,1.0,1.0,1.0
Decision Tree,1.0,1.0,1.0,1.0,1.0
Radom Forest,1.0,1.0,1.0,1.0,1.0
Adaboost,1.0,1.0,1.0,1.0,1.0
GaussionNB,1.0,1.0,1.0,1.0,1.0
Nueral Network,1.0,1.0,1.0,1.0,1.0


In [79]:
len(A)


8

In [80]:
len(B)

7

In [75]:
df2 = pd.concat([df,df1])

In [76]:
df2

Train Accuracy  Test Accuracy  Precision  \
Breast Cancer Logistic Regression             1.0            0.8        0.7   
              SVM                             1.0            0.8        0.7   
              Rnadom Forest                   1.0            0.8        0.7   
Diabetic      Logistic Regression             1.0            0.8        0.7   
              SVM                             1.0            0.8        0.7   
              Rnadom Forest                   1.0            0.8        0.7   
Breast Cancer Logistic Regression             1.0            0.8        0.7   
              SVM                             1.0            0.8        0.7   
              Rnadom Forest                   1.0            0.8        0.7   
Diabetic      Logistic Regression             1.0            0.8        0.7   
              SVM                             1.0            0.8        0.7   
              Rnadom Forest                   1.0            0.8        0.7   

                                   Recall  F1 score  
Breast Cancer Logistic Regression     0.5       0.6  
              SVM                     0.5       0.6  
              Rnadom Forest           0.5       0.6  
Diabetic      Logistic Regression     0.5       0.6  
              SVM                     0.5       0.6  
              Rnadom Forest           0.5       0.6  
Breast Cancer Logistic Regression     0.5       0.6  
              SVM                     0.5       0.6  
              Rnadom Forest           0.5       0.6  
Diabetic      Logistic Regression     0.5       0.6  
              SVM                     0.5       0.6  
              Rnadom Forest           0.5       0.6

In [32]:
d = {'A':['Breast Cancer','Breast Cancer','Breast Cancer','Diabetic','Diabetic','Diabetic'],
      'B':['Logistic Regression','SVM','Rnadom Forest''Logistic Regression','SVM','Rnadom Forest'],
        'C':['Train Accuracy','Test Accuracy','Precision','Recall','F1 score']}

In [ ]:
precision_score()

In [30]:
df.head()

,0,1,2,3,4
0,1.0,0.8,0.7,0.5,0.6
1,1.0,0.8,0.7,0.5,0.6
2,1.0,0.8,0.7,0.5,0.6
3,1.0,0.8,0.7,0.5,0.6
4,1.0,0.8,0.7,0.5,0.6


In [16]:
df.head()

,D
0,"[1, 1, 1, 1, 1, 1]"
1,"[0.5, 1.0, 1.0, 1.0, 1.0, 1.0]"
2,"[1.0, 0.8, 1.0, 1.0, 1.0, 1.0]"
3,"[1.0, 1.0, 0.7, 1.0, 1.0, 1.0]"
4,"[1.0, 1.0, 1.0, 1.0, 0.6, 1.0]"


In [21]:
data = loaddata()
X_train,X_test,y_train,y_test,X,y = preprocess()
f1scores = trainmodels()
EvaluateModels(f1scores)

Before removing outliers , rows -  568
After removing outliers , rows - 494
Number of records deleted -  74
Normalization done
Logistic Regression  report
-------------------------------------
 
[[110   2]
 [  6  53]]
              precision    recall  f1-score   support

           0       0.95      0.98      0.96       112
           1       0.96      0.90      0.93        59

    accuracy                           0.95       171
   macro avg       0.96      0.94      0.95       171
weighted avg       0.95      0.95      0.95       171

 
-------------------------------------
 
KNN  report
-------------------------------------
 
[[108   4]
 [  8  51]]
              precision    recall  f1-score   support

           0       0.93      0.96      0.95       112
           1       0.93      0.86      0.89        59

    accuracy                           0.93       171
   macro avg       0.93      0.91      0.92       171
weighted avg       0.93      0.93      0.93       171

 
---------

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: Conve

Nueral Network  report
-------------------------------------
 
[[88 24]
 [ 5 54]]
              precision    recall  f1-score   support

           0       0.95      0.79      0.86       112
           1       0.69      0.92      0.79        59

    accuracy                           0.83       171
   macro avg       0.82      0.85      0.82       171
weighted avg       0.86      0.83      0.83       171

 
-------------------------------------
 
        Model Results    
 --------------------------- 
Logistic Regression  : f1 score -  0.9298245614035089
K Nearest Neighbours  : f1 score -  0.8947368421052632
Support Vector Machine  : f1 score -  0.33707865168539325
Decision Tree  : f1 score -  0.8983050847457628
Random Forest  : f1 score -  0.9298245614035089
AdaBoost  : f1 score -  0.9572649572649573
Guassian Naive Bayes  : f1 score -  0.9122807017543859
Nueral Network  : f1 score -  0.7883211678832116
 -----------------------------
Best model
AdaBoost  : f1 score -  0.957264957264957